This is based off of someones code to analysis music type

#https://github.com/Harbim001/Analysing_Nigerian_Music_Growth_Globally/blob/main/analysing_nigerian_music_growth_globally%20(1).py


In [50]:
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Conv1D, GlobalMaxPooling1D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical

import pickle

import pandas as pd
from pathlib import Path

dictionary_path = "/Users/cslinxs/Desktop/all_weeks/summer24-lab/measurements_auscultatory_dictionary_ALL.pkl"

with open(dictionary_path, 'rb') as handle:
    all_auscultatory_dic = pickle.load(handle)


# Getting kid of ppl with bad averages , only 9 ppl with in dataset
bad_keys = []
for key, each_grouping in all_auscultatory_dic.items():
    if True in list(np.isnan(each_grouping[0])):
        bad_keys.append(key)
print(len(bad_keys))

for key in bad_keys:
    del all_auscultatory_dic[key]



#This groups ppl and would be usedful if I don't want overlaps in the people
group_ppl = {}
for key, each_grouping in list(all_auscultatory_dic.items()):
    person_id = key.split("_")[1].split("/")[1]
    # if person_id in list(people_htn):
    if not(person_id in group_ppl):
        group_ppl[person_id] = []
    group_ppl[person_id].append({key: each_grouping})
         


9


First im going to split ppl if they have hyper tension or not

In [56]:
# This gets the person
path_aurora_dataset = Path('/Users/cslinxs/Desktop/aurora_dataset/aurora_dataset/')

# This is getting the PID of ppl with Hypertenstion
participants_data =  path_aurora_dataset / "participants.tsv"
participants_csv_data = pd.read_csv(participants_data,sep = '\t')
participants_csv_htn = participants_csv_data[participants_csv_data["self_report_htn"].isin(["managed","unmanaged"])]
participants_csv_htn = participants_csv_htn[participants_csv_htn["pid"].str.contains('a')]
people_htn = participants_csv_htn["pid"]
people_htn


def get_person_data(PID):
    return ([([each_data[0] for each_data in item.values()]) for item in group_ppl[PID]], [([each_data[1] for each_data in item.values()][0]) for item in group_ppl[PID]], [item.keys() for item in group_ppl[PID]])

# This groups people by a feature
def get_person_data_group(diction, identifier):
    return ([([each_data[0] for each_data in item.values()]) for item in diction[identifier]], [([each_data[1] for each_data in item.values()][0]) for item in diction[identifier]], [item.keys() for item in diction[identifier]])

def get_person_split_data(dictionary_with_people):
    return ([item[0] for item in dictionary_with_people],[item[1] for item in dictionary_with_people],[item[2] for item in dictionary_with_people])

def get_people_with_hypertenstion():
    hyperstion_people = []
    non_hypertension_people = []
    for key, each_grouping in list(all_auscultatory_dic.items()):
        person_id = key.split("_")[1].split("/")[1]
        if person_id in list(people_htn):
            hyperstion_people.append(each_grouping)
        else:
            non_hypertension_people.append(each_grouping)
            
    return (hyperstion_people,non_hypertension_people)


In [61]:
people_with_hypertenstion = get_people_with_hypertenstion()[0]
people_with_out_hypertenstion = get_people_with_hypertenstion()[1]

# for now I will only test on people with hyptertension
X = get_person_split_data(people_with_hypertenstion)[0]
y = get_person_split_data(people_with_hypertenstion)[1]


In [62]:
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [63]:
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)
y = to_categorical(y)

/Users/cslinxs/Library/Python/3.9/lib/python/site-packages/sklearn/preprocessing/_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [64]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [65]:
X_train = np.expand_dims(X_train, axis=2)
X_test = np.expand_dims(X_test, axis=2)


In [67]:
def create_model():
    model = Sequential()
    model.add(Conv1D(64, kernel_size=3, activation='relu', input_shape=(X_train.shape[1], 1)))
    model.add(Conv1D(128, kernel_size=3, activation='relu'))
    model.add(GlobalMaxPooling1D())
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(y_train.shape[1], activation='softmax'))

    model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

    return model

model = create_model()

In [68]:
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.1)

test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f"Test accuracy: {test_accuracy}")

Epoch 1/50
90/90 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.0102 - loss: 5.4671 - val_accuracy: 0.0063 - val_loss: 5.1762
Epoch 2/50
90/90 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.0120 - loss: 5.1461 - val_accuracy: 0.0125 - val_loss: 5.1182
Epoch 3/50
90/90 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.0138 - loss: 5.0853 - val_accuracy: 0.0094 - val_loss: 5.1093
Epoch 4/50
90/90 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.0176 - loss: 5.0430 - val_accuracy: 0.0063 - val_loss: 5.1186
Epoch 5/50
90/90 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.0182 - loss: 5.0417 - val_accuracy: 0.0094 - val_loss: 5.1138
Epoch 6/50
90/90 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.0168 - loss: 5.0021 - val_accuracy: 0.0094 - val_loss: 5.1140
Epoch 7/50
90/90 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.0159 - loss: 5.0155 - val_accuracy: 0.0188 - val_loss: 5.1115
Epoch 8/50
90/90 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.0162 - loss: 5.0053 - val_accuracy: 0.0188 - val_loss:

In [ ]:
# Standardize the feature data using the same scaler used for the training data
X_new = scaler.transform(new_data)

# Reshape the input data for 1D CNN
X_new = np.expand_dims(X_new, axis=2)

# Predict the probabilities for each genre
predictions = model.predict(X_new)

# Get the genre index with the highest probability for each sample
predicted_indices = np.argmax(predictions, axis=1)

# Decode the genre indices to genre names
predicted_genres = label_encoder.inverse_transform(predicted_indices)

# Add the predicted genres to the new dataset
new_data['predicted_genre'] = predicted_genres

print(new_data)

print(tabulate(new_data.head(), headers='keys', tablefmt='pretty'))